In [ ]:
import numpy as np
import backtrader as bt

class FMS_Strategy(bt.Strategy):
    params = (
        ('init_short_ratio', 10.),
        ('profit_goal', 0.3),
        ('multiplier_limit', 2**10),
    )
    
    def log(self, txt, dt=None):
        dt = dt or self.data0.datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        #prediction on 2023-12-17 from models (ratio of close prices)
        self.dic_pred = {'SPX': [1.0042133331298828,
                          1.0074454545974731,
                          1.0086519718170166,
                          1.0071972608566284,
                          1.0062665939331055,
                          1.0049595832824707,
                          1.0041393041610718,
                          1.0084354877471924,
                          1.0064897537231445,
                          1.0067540407180786,
                          1.0062111616134644,
                          1.0052202939987183,
                          1.0046688318252563,
                          1.0035264492034912,
                          1.0025644302368164],
                         'NKX': [1.001426339149475,
                          0.9991018772125244,
                          1.001373052597046,
                          0.9996775984764099,
                          1.0001791715621948,
                          1.0002129077911377,
                          0.9998658299446106,
                          0.9988505244255066,
                          0.9992351531982422,
                          1.0025736093521118,
                          1.0017427206039429,
                          1.0016067028045654,
                          1.0021313428878784,
                          1.0018638372421265,
                          1.0015697479248047],
                         '2YUSY': [0.9996762275695801,
                          0.9973750114440918,
                          0.998511552810669,
                          0.9956330060958862,
                          0.9980739951133728,
                          1.0000396966934204,
                          1.0028233528137207,
                          1.0049209594726562,
                          1.0060358047485352,
                          1.0056099891662598,
                          1.0065584182739258,
                          1.0080634355545044,
                          1.0106127262115479,
                          1.0114061832427979,
                          1.012352705001831],
                         '10YUSY': [0.9968363046646118,
                          0.9985263347625732,
                          1.000723958015442,
                          1.0073760747909546,
                          1.007480263710022,
                          1.0074888467788696,
                          1.0092558860778809,
                          1.0073784589767456,
                          1.009335994720459,
                          1.0098748207092285,
                          1.0077608823776245,
                          1.010390043258667,
                          1.0130491256713867,
                          1.016015887260437,
                          1.0162909030914307],
                         'USDEUR': [1.0011792182922363,
                          1.0027763843536377,
                          1.0016621351242065,
                          1.0033330917358398,
                          1.003596544265747,
                          1.00363290309906,
                          1.003009557723999,
                          1.0023223161697388,
                          1.0025298595428467,
                          1.0021010637283325,
                          1.0026016235351562,
                          1.0026105642318726,
                          1.0022625923156738,
                          1.001440405845642,
                          1.0030477046966553],
                         'USDJPY': [0.9990713000297546,
                          1.0013352632522583,
                          1.0009260177612305,
                          1.0000416040420532,
                          0.9997223019599915,
                          1.0011886358261108,
                          1.0007007122039795,
                          0.9997785687446594,
                          0.9974855184555054,
                          0.9965397715568542,
                          0.997887372970581,
                          0.9985108375549316,
                          0.9997363686561584,
                          1.0015538930892944,
                          1.0022351741790771],
                         'BTCUSD': [1.003149390220642,
                          0.9980610013008118,
                          0.9893614649772644,
                          0.9891991019248962,
                          0.9921113848686218,
                          0.9909883141517639,
                          0.9927812814712524,
                          0.9953610897064209,
                          0.9972026944160461,
                          0.9938741326332092,
                          0.9904754161834717,
                          0.9894970655441284,
                          0.9898374080657959,
                          0.9865140914916992,
                          0.9872955679893494],
                         'ETH': [0.9912291169166565,
                          0.9897580742835999,
                          0.9864181876182556,
                          0.9895048141479492,
                          0.9805629253387451,
                          0.977593183517456,
                          0.9749253392219543,
                          0.9727044105529785,
                          0.9674498438835144,
                          0.9634069204330444,
                          0.9583613276481628,
                          0.9615297913551331,
                          0.9667393565177917,
                          0.9674011468887329,
                          0.963093638420105]}
        
        #determine short asset
        min_val = 2
        for key, value in self.dic_pred.items():
            if value[-1]<min_val:
                min_val = value[-1]
                short_key = key
        self.dic_pred.pop(short_key)
        
        self.short_asset = short_key
        print(f'short: {self.short_asset}')
        
        self.multiplier = 2
        self.n_day = 0
        self.init_flag = True
        self.init_cash = self.broker.get_cash()
        self.stop_flag = False


    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('buy order,%s, %.2f, %i' % (order.data._name,
                                                order.executed.price, order.executed.size))
            elif order.issell():
                self.log('sell order, %s, %.2f, %i' % (order.data._name,
                                                 order.executed.price, order.executed.size))

#     def notify_trade(self, trade):
#         if trade.isclosed:
#             print('Gross Profit %0.2f, Profit After Commission % 0.2f, Commission %.2f, Market Value %.2f' %
#                   (trade.pnl, trade.pnlcomm, trade.commission, self.broker.getvalue()))
                   
    def myShortSell(self):
        # choose the short asset at start
        for d in self.datas:
            if d._name == self.short_asset:
                size = int(self.init_cash/d.close[0])*self.params.init_short_ratio*(self.multiplier/2)
                print(f'short size:', size)
                self.sell(size=size,data=d._name, exectype=bt.Order.Market)
                if self.multiplier < self.params.multiplier_limit:
                    self.multiplier *= 2
                #self.order_target_percent(target=-self.params.init_short_ratio, data=d._name)
                
            
    def next(self):
        
        #stop
        if self.stop_flag:
            for d in self.datas:
                size = self.getposition(d).size
                if size > 0:
                    self.sell(size=size, data=d._name, exectype=bt.Order.Market)
                elif size < 0:
                    self.buy(size=size, data=d._name, exectype=bt.Order.Market)
            return
        
        #profit_goal checking
        earning_ratio = (self.broker.getvalue()/self.init_cash-1)
        if earning_ratio > self.params.profit_goal:
            self.stop_flag = True
            
            for d in self.datas:
                size = self.getposition(d).size
                if size > 0:
                    self.sell(size=size, data=d._name, exectype=bt.Order.Market)
                elif size < 0:
                    self.buy(size=size, data=d._name, exectype=bt.Order.Market)
            print(f'stopped')
            return
        
        print('day',self.n_day)
        
        #short
        if self.init_flag:
            self.myShortSell()
            self.init_flag = False
            
        #martingale short
        elif self.last_value > self.broker.getvalue():
            self.myShortSell()
                    
        #determine long asset
        max_val = -1
        for key, value in self.dic_pred.items():
            if self.n_day == 0:
                r = value[0]
            else:
                r = value[self.n_day]/value[self.n_day-1]
            if r > max_val:
                max_val = r
                self.long_asset = key
        
        #use all cash available for long
        print('long target', self.long_asset)
        for d in self.datas:
            if d._name == self.long_asset:
                size = int(self.broker.get_cash()/d.close[0])
                print(f'buy size:',size)
                self.buy(size=size,data=d._name)
              
        self.last_value = self.broker.getvalue()
        if self.n_day < 14:
            self.n_day += 1
        

## For testing 

In [ ]:
import pf_api
dic_code_bt = pf_api.PFData(codes='SPX,NKX,2YUSY,10YUSY,USDEUR,USDJPY,BTCUSD,ETH', start='2023-04-20', end='2023-05-01')

cerebro = bt.Cerebro()
startcash = 10000000
cerebro.broker.setcash(startcash)
cerebro.broker.setcommission(0.0003)

for code, datafeed in dic_code_bt.items():
    cerebro.adddata(datafeed, name=code) # add datafeed with data name

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.addstrategy(FinalCompetitionStrategy)
cerebro.run()
print('End Portfolio Value: %.2f' % cerebro.broker.getvalue()) 
print(f'Profit: {cerebro.broker.getvalue()/startcash-1:.4f}')